In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

tb_logdir = "/tmp/tf_tutorial/06_ConvNeuralNetwork"

#Data parameters
image_size = 28*28
n_class = 10

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
#Train parameters
learning_rate = 0.1
total_steps = 500
batch_size = 100

X = tf.placeholder(tf.float32, [None, image_size], name='InputImage')
Y = tf.placeholder(tf.float32, [None, n_class], name='Label')

In [3]:
def add_conv_layer(self, n_filter, k, name):
    self = tf.layers.conv2d(self, filters=n_filter, kernel_size=k)
    return self

def add_max_pool_layer(self, p, name):
    self = tf.layers.max_pooling2d(self, pool_size=p, strides=1, name=name)
    return self

def add_dense_layer(self, units, name):
    self = tf.layers.dense(self, units=units)
    return self
    
def build_cnn(input_layer):
    with tf.name_scope('CNN'):
        net = add_conv_layer(input_layer, 64, (3,3),'Conv_1')
        net = add_max_pool_layer(net, 3, 'Max_1')
        net = add_conv_layer(net, 32, (3,3),'Conv_2')
        net = add_max_pool_layer(net, 3, 'Max_2')
    return net

def flatten_layer(input_net):
    net = tf.contrib.layers.flatten(input_net)
    #---raw version:
    '''
    with tf.name_scope('Flatten'):
        product = np.prod(np.asarray(input_net.shape.as_list()[1:]))
        net = tf.reshape(input_net, [-1, product])
    '''
    return net
        
        
def build_dnn(input_net):
    with tf.name_scope('DNN'):
        net = add_dense_layer(input_net, 32, 'Dense_1')
        net = add_dense_layer(net, n_class, 'Dense_2')
    return net

In [4]:
with tf.name_scope('Model'):
    input_layer = tf.reshape(X, [-1, 28, 28, 1])
    cnn = build_cnn(input_layer)
    cnn = flatten_layer(cnn)
    fcnn = build_dnn(cnn)

In [5]:
with tf.name_scope('Loss'):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fcnn, labels=Y))
with tf.name_scope("Adam"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
with tf.name_scope('Accuracy'):
    correct_pred =  tf.equal(tf.argmax(fcnn, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
input_image = tf.reshape(X, [-1, 28, 28, 1])
tf.summary.image("input_image", input_image)
tf.summary.scalar('loss', loss_op)
tf.summary.scalar('accuracy', accuracy)
merge_summary_op = tf.summary.merge_all()

In [6]:
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(tb_logdir + '/train', graph=sess.graph)
    test_writer = tf.summary.FileWriter(tb_logdir + '/test')
    for step in range(total_steps):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        summary, acc, _ = sess.run([merge_summary_op, accuracy, train_op], feed_dict={X:batch_x, Y:batch_y})
        train_writer.add_summary(summary, step)
        summary = sess.run(merge_summary_op, feed_dict={X:mnist.test.images[:3000], Y:mnist.test.labels[:3000]})
        test_writer.add_summary(summary, step)
        if step%100 == 0:
            print("step: {}  accuracy:{:.2f}%".format(step, acc*100))
    print('Training Complete!')

step: 0  accuracy:12.00%
step: 100  accuracy:81.00%
step: 200  accuracy:94.00%
step: 300  accuracy:93.00%
step: 400  accuracy:97.00%
Training Complete!
